In [33]:
import pandas as pd
import json
from tqdm import tqdm
import ast
import numpy as np


In [34]:
def str2list(x):
    if len(x) > 0:
        if x[0] != '[':
            list = [x]
        else: 
            list = ast.literal_eval(x)
            if len(list) == 0: list = ['Empty']
    else: list = ['Empty']

    return list

In [35]:
basic_info = pd.read_csv('/opt/ml/wine/data/basic_info_total.csv', encoding='utf-8-sig')
with open('/opt/ml/wine/code/feature_map/item2idx.json','r') as f:
    item2idx = json.load(f)

<ipython-input-35-90518a68fd11>:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  basic_info = pd.read_csv('/opt/ml/wine/data/basic_info_total.csv', encoding='utf-8-sig')


In [36]:
item_data = pd.read_csv('/opt/ml/wine/server/data/item_df_allfeature.csv', encoding='utf-8-sig')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
wine_df = pd.read_csv('/opt/ml/wine/data/wine_df.csv', encoding='utf-8-sig')

In [ ]:
wine_df['grape'].fillna('[]', inplace = True)
wine_df['grape'] = wine_df['grape'].apply(lambda x: str2list(x))

In [ ]:
wine_df['item_id'] = wine_df['url'].map(item2idx)
wine_df = wine_df[wine_df['item_id'].isna()==False]
wine_df['item_id'] = wine_df['item_id'].astype(int).astype('category')

In [ ]:
basic_info['item_id'] = basic_info['url'].map(item2idx)
basic_info = basic_info[basic_info['item_id'].isna()==False]
basic_info['item_id'] = basic_info['item_id'].astype(int).astype('category')

In [ ]:
basic_info.set_index('item_id', inplace= True)
item_data.set_index('item_id', inplace= True)

In [ ]:
no_key = 0
for item_id in tqdm(item_data.index):
    try:
        item_data.loc[item_id,'winery'] = basic_info.loc[item_id,'winery']
        item_data.loc[item_id,'grape'] = basic_info.loc[item_id,'grapes']
        item_data.loc[item_id,'country'] = basic_info.loc[item_id,'country']
        item_data.loc[item_id,'region'] = basic_info.loc[item_id,'region1']
        item_data.loc[item_id,'region2'] = basic_info.loc[item_id,'region2']
        item_data.loc[item_id,'region3'] = basic_info.loc[item_id,'region3']
        item_data.loc[item_id,'region4'] = basic_info.loc[item_id,'region4']
        item_data.loc[item_id,'wine_style'] = basic_info.loc[item_id,'wine_style']
        item_data.loc[item_id,'allergens'] = basic_info.loc[item_id,'allergens']
        item_data.loc[item_id,'alcohol content'] = basic_info.loc[item_id,'alcohol content']
    except KeyError:
        try:
            item_data.loc[item_id,'grape'] = wine_df.loc[item_id,'grape']
        except:
            no_key +=1
print(no_key)

100%|██████████| 77834/77834 [04:15<00:00, 304.55it/s] 

19697


In [ ]:
item_data.rename(columns = {'region':'region1'}, inplace= True)

In [ ]:
item_data['item_id'] = item_data.index

In [ ]:
item_data.to_csv('/opt/ml/wine/data/item_df_allfeature.csv',index = False, encoding='utf-8-sig')

In [ ]:
with open('/opt/ml/wine/data/sample_vectors.json','r') as f: 
    sample_vec = json.load(f)

In [ ]:
vectors = []
for id in tqdm(item_data.item_id):
    if id in sample_vec.keys():
        vectors.append(np.array(sample_vec[id]))
    else:
        vectors.append(None)
item_data['vectors'] = vectors

100%|██████████| 77834/77834 [00:00<00:00, 363998.64it/s]


In [ ]:
import re

def keep_only_english(text):
    try:
        english_text = re.sub(r'[^a-zA-Z\s]', '', text)
        return english_text.lower().strip()
    except: return None

In [ ]:
for col in ['country','region1', 'winetype', 'wine_style']:
     item_data[col] = item_data[col].apply(keep_only_english)

In [ ]:
vector_item = item_data[item_data.vectors.isna()==False]

In [ ]:
grouped_vectors = vector_item.groupby(['country','region1', 'winetype', 'wine_style'] ).agg({
    'vectors': 'mean'
}).reset_index()

In [ ]:
from collections import deque
from pandas import DataFrame

def find_vectors(columns_name : deque, grouped_vectors: DataFrame):
    column, name = columns_name.popleft()
    if name is not None:
        grouped_vectors = grouped_vectors.query(f"{column} == '{name}'")
        if columns_name:
            vector = find_vectors(columns_name, grouped_vectors.drop(column,axis = 1))
        else: 
            vector = grouped_vectors.vectors.mean()
            
        return vector
    else:
        return grouped_vectors.vectors.mean()
    

In [ ]:
non_vectors = item_data[item_data['vectors'].isna()==True]
non_vectors_cols = list(grouped_vectors.columns)
non_vectors_cols.append('item_id')
non_vectors = non_vectors.loc[:, non_vectors_cols]


In [37]:
vectors = []
for index, row in tqdm(non_vectors.iterrows()):
    columns_name = deque()
    import pdb
    for column, name in zip(row.keys(), row.values):
        if column != 'vectors':
            columns_name.append((column, name))
        else: break
    vector = find_vectors(columns_name, grouped_vectors)
    vectors.append(vector)
   

4778it [01:21, 59.73it/s]